In [28]:
# Mounting to drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
# Assigning Dataset Path
%cd '/content/drive/MyDrive/Colab Notebooks/Dataset'
%ls
dataset = 'car-dataset.csv'
#df1 = pd.read_csv('dataset')

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/Dataset'
/content
drive/  sample_data/


**Import necessary libraries**

In [30]:
import math #mathematical computation
import pandas as pd #dataframe
import numpy as np #mathematical computations
import matplotlib.pyplot as plt #visualization
import matplotlib
import seaborn as sns #visualization
import json #exporting columns
import pickle #saving the model
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression #Linear Regression
from sklearn.linear_model import Lasso #Lasso Regression
from sklearn.tree import DecisionTreeRegressor #Decision Tree Regression
from sklearn.ensemble import RandomForestRegressor #Random Forest Regression
from sklearn.model_selection import train_test_split #Splitting the dataset into training and testing
from sklearn.model_selection import ShuffleSplit #Random shuffling
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score #Score cross validation
from xgboost import XGBRegressor # XGBoost Regression
from sklearn.model_selection import GridSearchCV #Hyper parameter tuning
from sklearn.preprocessing import MinMaxScaler # Min Max Scaler
from warnings import simplefilter #Filtering warnings
import plotly.express as px






simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [31]:
from sklearn.model_selection import train_test_split #Splitting the dataset into training and testing
from sklearn.model_selection import ShuffleSplit #Random shuffling

In [33]:
#Read the CSV file and Load the top 10 instances
df1 = pd.read_csv(dataset)
#df1.head(10)

FileNotFoundError: ignored

In [ ]:
#Load the bottom 10 instances 
df1.tail(10)

**Load data into DataFrame**

In [ ]:
#shape
df1.shape


In [ ]:
# data columns
df1.columns

In [ ]:
#Return a Series containing counts of unique rows in the DataFrame.
df1['name'].value_counts()

**Dropping feature that is not required in building our model**

In [ ]:
# I also consider reindexing the columns, putting the dependent variable at the end
# Dropping the name of the car and seller type
# Retained the columns transmission,number of previous owner, fuel, year and kilometer driven for these features might greatly affect the selling price

df2 = df1.drop(['name','seller_type'],axis=1).reindex(['transmission','fuel','owner','year','km_driven','selling_price'],axis=1)
df2.head()

**Check for null values**

In [ ]:
# method 1
df2.isnull().sum()

In [ ]:
# method 2
df2.isnull().any()

In [ ]:
#drop null elements
df2 = df2.dropna()

In [ ]:
# Obtain the summary of the dataframe
df2.info()

In [ ]:

# Generating the Strongly correlated values
df_num_corr = df1.corr()['selling_price'][:-1] 
golden_features_list = df_num_corr[abs(df_num_corr) > 0.5].sort_values(ascending=False)
print("There is {} strongly correlated values with Selling Price:\n{}".format(len(golden_features_list), golden_features_list))
 #From seaborn documentation
hm = df2[df2.columns].corr()
g = sns.heatmap(hm, annot = True, annot_kws={'size':9}, linewidths = 2)

**Check for data consistency / Explaratory Data Analysis**

In [ ]:
df2.owner.unique()

In [ ]:
df2.fuel.unique()

In [ ]:
df2.transmission.unique()

In [ ]:
df2 ['transmission'].value_counts()

In [ ]:
df2.year.unique()

**Adding new feature**

In [ ]:
# adding a converted price for bdt

df2['selling_price_bdt'] = round(df2['selling_price'] * 1.17,2)

# dropping selling price in rupee

df3 = df2.drop('selling_price',axis=1)
df3.head()

**Exploratory Data Analysis**

In [ ]:

fig = px.histogram (df3, x ='year', color = 'transmission',
                    labels={'year':'Year of manufacture'},
                    title = "manufacturing year and fuel" )

fig.show()

The figure above shows a histogram containing values of 2 different columns. From the figurev the relationship between manufactering year and fuel can be observed.

In [ ]:
#Manual and Automatic Cars

sns.pairplot(df3,hue = 'transmission',diag_kind = "kde",kind = "scatter",palette = "husl",height=3.5)
plt.show()


We can see that automatic cars have higher price range than manual type cars though the distribution of automatic cars were skewed to the right. We can also see the increase of automatic cars between 2015 and 2020. In this instance, we can clearly see some outliers in selling price and kilometers driven.

**Fuel types**

In [ ]:
df3['fuel'].value_counts()

Since most of the data points are Diesel and Petrol, we can label other fuel types as Others.

In [ ]:
df4 = df3
df4.fuel = df4.fuel.map(lambda x: x if x in ['Diesel','Petrol'] else 'Other')
df4.fuel.unique()

In [ ]:
manual = df4[df4['transmission']=='Manual']
automatic = df4[df4['transmission']=='Automatic']

In [ ]:
print('Manual type car')
sns.pairplot(manual,hue = 'fuel',diag_kind = "kde",kind = "scatter",palette = "husl",height=3.5)
plt.show()

In [ ]:
print('Automatic type car')
sns.pairplot(automatic,hue = 'fuel',diag_kind = "kde",kind = "scatter",palette = "husl",height=3.5)
plt.show()

We can see that the selling price of diesel type cars in both manual and automatic were more spread than petrol and other fuel hence getting higher average and range of selling price.

Owner

In [ ]:
df4.groupby('owner')[['km_driven','selling_price_bdt']].agg(['count','mean']).applymap(lambda x: format(x,'.0f'))

In [ ]:
# we will be dropping test drive cars and will merge third owner and 
# forth & above owner as third owner and above

df5 = df4[~(df4['owner']=='Test Drive Car')]
df5.owner = df5.owner.map(lambda x: x if x in ['First Owner','Second Owner'] else 'Third Owner & Above')

df5['owner'].value_counts()


In [ ]:
sns.pairplot(df5,hue = 'owner',diag_kind = "kde",kind = "scatter",palette = "husl",height=3.5)
plt.show()

Cars sold by first owner have higher prices than the rest. Test drive cars tends to have a higher price s as well though there is a distinction between test drive cars in manual and automatic.

Year and Kilometer Driven

In [ ]:
km_mean = df5.groupby('year')['km_driven'].mean()

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(16,8))

ax[0].bar(km_mean.index,km_mean)
sns.distplot(manual['km_driven'],ax=ax[1])
sns.distplot(automatic['km_driven'],ax=ax[1])

ax[0].set_title('Average kilometer driven each year')
ax[0].set_xlabel('Manufactering Year')
ax[0].set_ylabel('Kilometer Driven')

ax[1].set_title('Kilometers driven distribution')
ax[1].legend(['Manual','Automatic'])

plt.show()

We can see that average kilometers driven rises up from 1995 until 2005 and linearly goes down until 2020. We can also see some outliers present in the distribution plot.

Year and Selling Price

In [ ]:
year_mean_manual = df5[df5['transmission']=='Manual'].groupby('year')['selling_price_bdt'].mean()
year_mean_automatic = df5[df5['transmission']=='Automatic'].groupby('year')['selling_price_bdt'].mean()

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(16,4))

ax[0].bar(year_mean_manual.index,year_mean_manual)
ax[1].bar(year_mean_automatic.index,year_mean_automatic)

ax[0].set_title('Average Selling Price of Manual Cars every Year')
ax[0].set_xlabel('Year')
ax[0].set_ylabel('Selling Price')

ax[1].set_title('Average Selling Price of Automatic Cars every Year')
ax[1].set_xlabel('Year')
ax[1].set_ylabel('Selling Price')

plt.show()

We can see that selling price of manual cars grows linearly each year whereas automatic cars have wavy averages in each year but we can clearly see that selling price grows linearly as well.

Removing outliers

In [ ]:
df5.groupby('transmission').agg(['mean','std','min','max']).applymap(lambda x: format(x,'.0f')).drop(['year'],axis=1)

Clearly we have some outliers present in kilometers driven and selling price. We need to remove these outliers by using IQR method.

In [ ]:
def remove_outlier_km_driven(df):
    temp = pd.DataFrame()
    
    df_km_driven = df['km_driven']
    Q1 = df_km_driven.quantile(0.25)
    Q3 = df_km_driven.quantile(0.75)
    IQR = Q3 - Q1
    df_outlier = df_km_driven[(df_km_driven < (Q1 - 1.5 * IQR)) | (df_km_driven > (Q3 + 1.5 * IQR))]
    temp = pd.concat([temp,df_outlier])
        
    return df.drop(temp.index)

df6 = remove_outlier_km_driven(df5)

Removing outliers in selling price would be separated by transmission type and will be done in each year

In [ ]:
def remove_outlier_selling_price(df):
    temp = pd.DataFrame()
    for year in sorted(df.year.unique()):
        year_price_manual = df[(df['year']==year) & (df['transmission']=='Manual')]['selling_price_bdt']
        manual_Q1 = year_price_manual.quantile(0.25)
        manual_Q3 = year_price_manual.quantile(0.75)
        manual_IQR = manual_Q3 - manual_Q1
        outlier_manual = year_price_manual[(year_price_manual < (manual_Q1 - 1.5 * manual_IQR)) | (year_price_manual > (manual_Q3 + 1.5 * manual_IQR))]
        temp = pd.concat([temp,outlier_manual])
        
        year_price_automatic = df[(df['year']==year) & (df['transmission']=='Automatic')]['selling_price_bdt']
        automatic_Q1 = year_price_automatic.quantile(0.25)
        automatic_Q3 = year_price_automatic.quantile(0.75)
        automatic_IQR = automatic_Q3 - automatic_Q1
        outlier_automatic = year_price_automatic[(year_price_automatic < (automatic_Q1 - 1.5 * automatic_IQR)) | (year_price_automatic > (automatic_Q3 + 1.5 * automatic_IQR))]
        temp = pd.concat([temp,outlier_automatic])
    return df.drop(temp.index)

df7 = remove_outlier_selling_price(df6)

In [ ]:
year_mean_manual = df7[df7['transmission']=='Manual'].groupby('year')['selling_price_bdt'].mean()
year_mean_automatic = df7[df7['transmission']=='Automatic'].groupby('year')['selling_price_bdt'].mean()

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(16,4))

ax[0].bar(year_mean_manual.index,year_mean_manual)
ax[1].bar(year_mean_automatic.index,year_mean_automatic)

ax[0].set_title('Average Selling Price of Manual Cars every Year')
ax[0].set_xlabel('Year')
ax[0].set_ylabel('Selling Price')

ax[1].set_title('Average Selling Price of Automatic Cars every Year')
ax[1].set_xlabel('Year')
ax[1].set_ylabel('Selling Price')

plt.show()

In [ ]:
# we can also safely remove the datapoints before 2005 since it produces inconsistency and the data points
# below 2000 have low value count
df8 = df7[df7['year']>2005]

In [ ]:
year_mean_manual_price = df8[df8['transmission']=='Manual'].groupby('year')['selling_price_bdt'].mean()
year_mean_automatic_price = df8[df8['transmission']=='Automatic'].groupby('year')['selling_price_bdt'].mean()
year_mean_manual_km = df8[df8['transmission']=='Manual'].groupby('year')['km_driven'].mean()
year_mean_automatic_km = df8[df8['transmission']=='Automatic'].groupby('year')['km_driven'].mean()

In [ ]:
fig,ax = plt.subplots(nrows=2,ncols=2,figsize=(16,8))

ax[0][0].bar(year_mean_manual_price.index,year_mean_manual_price)
ax[0][1].bar(year_mean_automatic_price.index,year_mean_automatic_price)

ax[1][0].bar(year_mean_manual_km.index,year_mean_manual_km)
ax[1][1].bar(year_mean_automatic_km.index,year_mean_automatic_km)

ax[0][0].set_title('Average Selling Price of Manual Cars every Year')
ax[0][0].set_ylabel('Selling Price')

ax[0][1].set_title('Average Selling Price of Automatic Cars every Year')
ax[0][1].set_ylabel('Selling Price')

ax[1][0].set_title('Average Kilometer driven of Manual Cars every Year')
ax[1][0].set_ylabel('Kilometer driven')

ax[1][1].set_title('Average Kilometer driven of Automatic Cars every Year')
ax[1][1].set_ylabel('Kilometer driven')

plt.show()

In [ ]:
def remove_outlier_owner(df):
    temp = pd.DataFrame()
    for a in sorted(df.year.unique()):
        for b in ['Manual','Automatic']:
            df_year = df[(df['year']==a)&(df['transmission']==b)]
            second_mean = df_year[df_year['owner']=='Second Owner']['selling_price_bdt'].mean()
            third_mean = df_year[df_year['owner']=='Third Owner & Above']['selling_price_bdt'].mean()
            df_outlier = df_year[((df_year['owner']=='Second Owner')&(df_year['selling_price_bdt']<third_mean)) | ((df_year['owner']=='First Owner')&(df_year['selling_price_bdt']<second_mean))]
            temp = pd.concat([temp,df_outlier])
    return df.drop(temp.index)
    
df9 = remove_outlier_owner(df8)

In [ ]:
def remove_outlier_last_year(df):
    temp = pd.DataFrame()
    for i in sorted(df.year.unique()):
        df_year = df[df['year']==i]
        km = df_year['km_driven']
        price = df_year['selling_price_bdt']
        outlier = df_year[(df_year['km_driven']<km.mean()-km.std())&(df_year['selling_price_bdt']<price.mean()-price.std()) | (df_year['km_driven']>km.mean()+km.std())&(df_year['selling_price_bdt']>price.mean()+price.std())]
        temp = pd.concat([temp,outlier])

    return df.drop(temp.index)
    
df11 = remove_outlier_last_year(df9)

In [ ]:
manual = df11[df11['transmission']=='Manual']
automatic = df11[df11['transmission']=='Automatic']

In [ ]:
sns.pairplot(df11,hue='transmission',diag_kind = "kde",kind = "scatter",palette = "husl",height=3.5)
plt.show()

As you can see in the scatter plot, kilometers driven decreases whereas selling price increases over the years. We can also see that automatic cars have higher prices than manual cars. Now it's time to build our model.

In [ ]:
df11.head()

In [ ]:
df11.shape

In [ ]:
transmission = pd.get_dummies(df11['transmission'])
fuel = pd.get_dummies(df11['fuel'])
owner = df11['owner'].map(lambda x: 1 if x=='First Owner' else 2 if x=='Second Owner' else 3)

X = pd.concat([transmission,fuel,owner,df11.drop(['transmission','fuel','owner','selling_price_bdt'],axis=1)],axis=1)
y = df11['selling_price_bdt']

In [ ]:
X.head(5)

In [ ]:
X.columns

In [ ]:
y.head(5)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
scale_vars = ['Automatic', 'Manual', 'Diesel', 'Other', 'Petrol', 'owner', 'year',
       'km_driven']
scaler = MinMaxScaler()
X_train[scale_vars] = scaler.fit_transform(X_train[scale_vars])
X_test[scale_vars] = scaler.transform(X_test[scale_vars])

In [ ]:
X_train.head()

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

Use K Fold cross validation to measure accuracy of our LinearRegression model.

In [ ]:
cv = ShuffleSplit(n_splits=5, test_size=0.15, random_state=0)

cross_val_score(LinearRegression(), X, y, cv=cv)

As you can see, we are getting low score from Linear Regression. Let's see if we can get better results from different regression models.

In [ ]:
from xgboost import XGBRegressor # XGBoost Regression

In [ ]:
def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        },
        'random_forest': {
            'model': RandomForestRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse','mae'],
                'n_estimators': range(1,50,5)
            }
        },
        'XGBoost': {
            'model': XGBRegressor(objective ='reg:squarederror'),
            'params': {
                'criterion' : ['mse','friedman_mse','mae'],
                #'objective' : 'reg:squarederror',
                'n_estimators': range(1,50,5),
                'max_depth': range(1,6,1),
                'eta' : [0.1,0.01],
                'colsample_bytree' : np.linspace(0,1,10,endpoint=False)
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=10)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

a = find_best_model_using_gridsearchcv(X,y)
a.head()

In [ ]:
a.values

In [ ]:
# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

# Scoring Parameters
scoring = ['r2','neg_mean_squared_error', 'neg_root_mean_squared_error', 'neg_median_absolute_error']
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=10)



# Linear Regression
linear_regression_score = cross_validate(LinearRegression(normalize= False), X, y, scoring=scoring, cv=cv)
print('Linear Regression\n')
print('The R2 Score  on the Testing dataset is:    ', linear_regression_score['test_r2'].mean())
print('The Log of Mean Squared Error on the Testing dataset is: ', math.log(-1 * linear_regression_score['test_neg_mean_squared_error'].mean()))
print('The Log of Root Mean Squared Error on the Testing dataset is: ', math.log(-1 * linear_regression_score['test_neg_root_mean_squared_error'].mean()))
print('The Log of Mean Absolute Error on the Testing dataset is:  ', math.log(-1 * linear_regression_score['test_neg_median_absolute_error'].mean()))


print('\n\n')


# Lasso Regresion
lasso_score = cross_validate(Lasso(alpha=2, selection='cyclic'), X, y, scoring=scoring, cv=cv)
print('Lasso Regression\n')
print('The R2 Score  on the Testing dataset is:    ', lasso_score['test_r2'].mean())
print('The Log of Mean Squared Error on the Testing dataset is: ', math.log(-1 * lasso_score['test_neg_mean_squared_error'].mean()))
print('The Log of Root Mean Squared Error on the Testing dataset is: ', math.log(-1 * lasso_score['test_neg_root_mean_squared_error'].mean()))
print('The Log of Mean Absolute Error on the Testing dataset is:  ', math.log(-1 * lasso_score['test_neg_median_absolute_error'].mean()))


print('\n\n')


# Decision Tree
decision_tree_score = cross_validate(DecisionTreeRegressor(criterion= 'mse',splitter = 'best'), X, y, scoring=scoring, cv=cv)
print('Decision Tree\n')
print('The R2 Score  on the Testing dataset is:    ', decision_tree_score['test_r2'].mean())
print('The Log of Mean Squared Error on the Testing dataset is: ', math.log(-1 * decision_tree_score['test_neg_mean_squared_error'].mean()))
print('The Log of Root Mean Squared Error on the Testing dataset is: ', math.log(-1 * decision_tree_score['test_neg_root_mean_squared_error'].mean()))
print('The Log of Mean Absolute Error on the Testing dataset is:  ', math.log(-1 * decision_tree_score['test_neg_median_absolute_error'].mean()))


print('\n\n')


# Random Forest
random_forest_score = cross_validate(RandomForestRegressor(criterion= 'friedman_mse',n_estimators=31), X, y, scoring=scoring, cv=cv)
print('Random Forest\n')
print('The R2 Score  on the Testing dataset is:    ', random_forest_score['test_r2'].mean())
print('The Log of Mean Squared Error on the Testing dataset is: ', math.log(-1 * random_forest_score['test_neg_mean_squared_error'].mean()))
print('The Log of Root Mean Squared Error  on the Testing dataset is: ', math.log(-1 * random_forest_score['test_neg_root_mean_squared_error'].mean()))
print('The Log of Mean Absolute Error on the Testing dataset is:  ', math.log(-1 * random_forest_score['test_neg_median_absolute_error'].mean()))



print('\n\n')


# XGBoost
xgboost_score = cross_validate(XGBRegressor(objective ='reg:squarederror',colsample_bytree = 0.8, criterion= 'mse', eta = 0.1, max_depth = 5, n_estimators = 46), X, y, scoring=scoring, cv=cv)
print('XGBoost\n')
print('The R2 Score  on the Testing dataset is:    ', xgboost_score['test_r2'].mean())
print('The Log of Mean Squared Error on the Testing dataset is: ', math.log(-1 * xgboost_score['test_neg_mean_squared_error'].mean()))
print('The Log of Root Mean Squared Error on the Testing dataset is: ', math.log(-1 * xgboost_score['test_neg_root_mean_squared_error'].mean()))
print('The Log of Mean Absolute Error on the Testing dataset is:  ', math.log(-1 * xgboost_score['test_neg_median_absolute_error'].mean()))




In [ ]:
Result = [
            {
                'Model': 'Linear Regression', 
                'R2 Score': 100 * linear_regression_score['test_r2'].mean(),
                'Log Mean Squared': math.log(-1 * linear_regression_score['test_neg_mean_squared_error'].mean()), 
                'Log Root Mean Squared': math.log(-1 * linear_regression_score['test_neg_root_mean_squared_error'].mean()),
                'Log Mean Absolute' : math.log(-1 * linear_regression_score['test_neg_median_absolute_error'].mean())
            },
            {
                'Model': 'Lasso Regression', 
                'R2 Score': 100 * lasso_score['test_r2'].mean(),
                'Log Mean Squared': math.log(-1 * lasso_score['test_neg_mean_squared_error'].mean()), 
                'Log Root Mean Squared': math.log(-1 * lasso_score['test_neg_root_mean_squared_error'].mean()),
                'Log Mean Absolute' : math.log(-1 * lasso_score['test_neg_median_absolute_error'].mean())
            },
            {
                'Model': 'Decision Tree', 
                'R2 Score': 100 *  decision_tree_score['test_r2'].mean(),
                'Log Mean Squared': math.log(-1 * decision_tree_score['test_neg_mean_squared_error'].mean()), 
                'Log Root Mean Squared': math.log(-1 * decision_tree_score['test_neg_root_mean_squared_error'].mean()),
                'Log Mean Absolute' : math.log(-1 * decision_tree_score['test_neg_median_absolute_error'].mean())
            },
            {
                'Model': 'XGBoost', 
                'R2 Score': 100 * xgboost_score['test_r2'].mean(),
                'Log Mean Squared': math.log(-1 * xgboost_score['test_neg_mean_squared_error'].mean()), 
                'Log Root Mean Squared': math.log(-1 * xgboost_score['test_neg_root_mean_squared_error'].mean()),
                'Log Mean Absolute' : math.log(-1 * xgboost_score['test_neg_median_absolute_error'].mean())
            },
            {
                'Model': 'Random Forest', 
                'R2 Score': 100 * random_forest_score['test_r2'].mean(),
                'Log Mean Squared': math.log(-1 * random_forest_score['test_neg_mean_squared_error'].mean()), 
                'Log Root Mean Squared': math.log(-1 * random_forest_score['test_neg_root_mean_squared_error'].mean()),
                'Log Mean Absolute' : math.log(-1 * random_forest_score['test_neg_median_absolute_error'].mean())
            }
        ] 
    

    
result_df = pd.DataFrame(Result) 
result_df.head(5)

In [ ]:
a4_dims = (11.7, 8.27)
fig, ax = plt.subplots(figsize=a4_dims)
sns.lineplot(ax=ax,data = result_df , x = "Model", y = "R2 Score")

In [ ]:
result_df = result_df.sort_values("R2 Score")
result_df.plot(x = "Model", y=['Log Mean Squared',"Log Root Mean Squared", 'Log Mean Absolute'], kind="bar",figsize=(10,10))

In [ ]:
xg_boost_model = XGBRegressor(objective ='reg:squarederror',colsample_bytree = 0.8, eta = 0.1, max_depth = 5, n_estimators = 46)

#train model
xg_boost_model.fit(X_train,y_train)
#print score
xg_boost_model.score(X_test,y_test)

In [ ]:
import joblib #saving the model
joblib.dump(xg_boost_model,'xg_boost_model.pkl')

In [ ]:
def get_price(transmission ,fuel_type, owner_type,  model_year  , km_driven):
    features =  ['Automatic', 'Manual', 'Diesel', 'Other', 'Petrol', 'owner', 'year','km_driven']
    fuel_dict = {'CNG': 0, 'Diesel': 1, 'Electric': 2, 'LPG': 3, 'Petrol': 4}
    owner_dict = {'First Owner': 0, 'Fourth & Above Owner': 1, 'Second Owner': 2, 'Test Drive Car': 3, 'Third Owner': 4}
    #fuel_type
    automatic = 0
    manual = 0

    # deciding fuel type
    Diesel = 0
    Other = 0
    Petrol = 0
    
    if fuel_type == 'Diesel':
        Diesel = 1
    elif fuel_type == 'Other':
        Other = 1
    elif fuel_type == 'Petrol':
        Petrol = 1


    # deciding transmission type
    if transmission == '1':
        automatic = 1
    else:
        manual = 1

    
    test_array = [ automatic , manual , Diesel, Other, Petrol, owner_dict[owner_type],  model_year, km_driven]
    
    test_array = np.array(test_array) # convert into numpy array
   
    test_array = test_array.reshape(1,-1) #reshape
    test_df = pd.DataFrame(test_array, columns = features)
  
    
    test_df[features] = scaler.transform(test_df[features])
    test_df[features] = scaler.inverse_transform(test_df[features])
 
    #declare path where you saved your model
    model_path = 'xg_boost_model.pkl'
    #open file
    file = open(model_path, "rb")
    #load the trained model
    trained_model = joblib.load(file)
    


    prediction = int(trained_model.predict(test_df))
    return prediction
   

In [ ]:
price = get_price('1','Petrol', 'First Owner', 2009 , 60000)
print(price)


In [ ]:
X_train.head()

In [ ]:
fuel_list = ['Petrol', 'Diesel', 'CNG', 'LPG', 'Electric']
for x in fuel_list:
  print('<option value="' + x + '">' + x + '</option>')

In [ ]:
owner_list = ['First Owner', 'Second Owner', 'Fourth & Above Owner','Third Owner', 'Test Drive Car']
for x in owner_list:
  print('<option value="' + x + '">' + x + '</option>')

In [ ]:
# Sort the list value in ascending order and then convert them into a python dictionary
def get_mapped_data():
    list_1 = ['First Owner', 'Second Owner', 'Fourth & Above Owner','Third Owner', 'Test Drive Car']
    list_2 = ['Petrol', 'Diesel', 'CNG', 'LPG', 'Electric']
    list_1.sort()
    list_2.sort()
    curr = 0
    mapping_1 = {}
    mapping_2 = {}
    for item in list_1:
        mapping_1[item] = curr
        curr = curr + 1;
    curr = 0
    for item in list_2:
        mapping_2[item] = curr
        curr = curr + 1;    
    
    return mapping_2

hell0 = get_mapped_data()
print(hell0)

In [ ]:
 fuel_dict = {'CNG': 0, 'Diesel': 1, 'Electric': 2, 'LPG': 3, 'Petrol': 4}
 print(fuel_dict['Diesel'])

In [ ]:
scaler_filename = "scaler.save"
joblib.dump(scaler, scaler_filename) 